325 Final Project: Nathaniel Gilbert, Emile Rebillard, Thomas Sorensen, Angelina Ponce

In [1]:
import pandas as pd

Introduction

We are data analysts employeed by the NCAA. We were given the assigment to study the given dataset to find any noticeable trends that should be researched further. We are going to focus on the Academic Progress Rate (APR), which is a calculation based off how many athletes on a team remain academiclly elligible and with the university. Most of the data within the NCAA dataset is not gathered by the NCAA. In fact, almost all of it is gathered by the member schools, and then shared with the NCAA. However, some of this data has been directly gathered by the NCAA. We will be attempting to answer the following four questions 
1. How have the retention rates for NCAA schools changed over the available data, from 2004 to 2019? We will be using all of the RET_Rate_XXXX columns where XXXX represents a different year for each column
2. Exploratory analysis: How many sports are fielded by each institution, and does that vary between NCAA divisions?  We will be using the SCL_UNITID and SCL_NAME columns as well as the Sport_Code and Sport_Name to analyse the counts of sports by each university. This is important to see if schools generally field the same number of sports or if there's drastic differences between schools and analyze what could be different from schools with a high number of sports and those with a low number.
3. What variables are the most positively correlated with multi-year APR, and can we accurately predict what a teams multi-year APR will be given other data? (regression). This will involve using linear regression on the whole dataset to determine which variables are significant.
4. Do Private or Public schools have better APR and academic eligibility rates? We will be using the multi-year APR and multi-year academic eligibility columns and SCL_PRIVATE column.

To start with though, we need to clean the dataset.

In [2]:
raw_data = pd.read_csv('C:/Users/natha/OneDrive/Documents/GitHub/DIDA_325_Final-project/csvs/raw data.csv')
drops=[0,12,20,85]
raw_data.drop(raw_data.columns[drops], axis=1, inplace=True)
# These columns were dropped because they were entirely empty.

cols_to_fill = list(range(82, 97)) # the columns we want to fill are the 83rd to 98th columns (0-indexed) so we are creating a list of their indices using the range function. 
raw_data.iloc[:, cols_to_fill] = raw_data.iloc[:, cols_to_fill].fillna(0)
# These columns were filled with 0s because some were empty but the documnetation said they should be 0s.

In [3]:
# Next we will see what columns are still empty.
# We will do this by using the isnull() method to check for null values and then summing them up with the sum() method.

null_counts = raw_data.isnull().sum()
null_counts = null_counts[null_counts > 0]
pd.set_option('display.max_columns', None)  #These two rows are used to set the display options for pandas so that all columns and rows are shown when printing the DataFrame.
pd.set_option('display.max_rows', None)

print(null_counts.sort_values(ascending=False))

D1_FB_CONF_19                     1578
ELIG_RATE_2004                     900
APR_RATE_2004_1000                 900
NUM_OF_ATHLETES_2004               900
RET_RATE_2004                      900
ELIG_RATE_2005                     867
APR_RATE_2005_1000                 867
RET_RATE_2005                      867
NUM_OF_ATHLETES_2005               867
APR_RATE_2006_1000                 711
ELIG_RATE_2006                     711
RET_RATE_2006                      711
NUM_OF_ATHLETES_2006               711
APR_RATE_2007_1000                 644
ELIG_RATE_2007                     644
RET_RATE_2007                      644
NUM_OF_ATHLETES_2007               644
APR_RATE_2008_1000                 576
ELIG_RATE_2008                     576
RET_RATE_2008                      576
NUM_OF_ATHLETES_2008               576
APR_RATE_2009_1000                 470
ELIG_RATE_2009                     470
RET_RATE_2009                      470
NUM_OF_ATHLETES_2009               470
APR_RATE_2010_1000       

In [4]:
#D1_FB_CONF_19 has a lot of null values, and is slightly redundant with CONFNAME_19 so we will drop it.
raw_data.drop(columns=['D1_FB_CONF_19'], inplace=True)
# Additionmally, any row with null values in the MULTIYR stats are useless to us so we will drop them as well.
clean_data=raw_data.dropna(subset=['MULTIYR_RET_RATE', 'MULTIYR_APR_RATE_1000_RAW', 'MULTIYR_APR_RATE_1000_CI', 'MULTIYR_APR_RATE_1000_OFFICIAL', 'MULTIYR_SQUAD_SIZE', 'MULTIYR_ELIG_RATE'])
# we use the subset paramter to specify which columns we want to check for null values and drop the rows with null values in those columns. https://www.w3schools.com/python/pandas/ref_df_dropna.asp



In [5]:
# This leaves us with the following dataset
clean_data.head()

,SCL_UNITID,SCL_NAME,SPORT_CODE,SPORT_NAME,ACADEMIC_YEAR,SCL_DIV_19,SCL_SUB_19,CONFNAME_19,SCL_HBCU,SCL_PRIVATE,MULTIYR_APR_RATE_1000_RAW,MULTIYR_APR_RATE_1000_CI,MULTIYR_APR_RATE_1000_OFFICIAL,RAW_OR_CI,MULTIYR_SQUAD_SIZE,MULTIYR_ELIG_RATE,MULTIYR_RET_RATE,APR_RATE_2019_1000,ELIG_RATE_2019,RET_RATE_2019,NUM_OF_ATHLETES_2019,APR_RATE_2018_1000,ELIG_RATE_2018,RET_RATE_2018,NUM_OF_ATHLETES_2018,APR_RATE_2017_1000,ELIG_RATE_2017,RET_RATE_2017,NUM_OF_ATHLETES_2017,APR_RATE_2016_1000,ELIG_RATE_2016,RET_RATE_2016,NUM_OF_ATHLETES_2016,APR_RATE_2015_1000,ELIG_RATE_2015,RET_RATE_2015,NUM_OF_ATHLETES_2015,APR_RATE_2014_1000,ELIG_RATE_2014,RET_RATE_2014,NUM_OF_ATHLETES_2014,APR_RATE_2013_1000,ELIG_RATE_2013,RET_RATE_2013,NUM_OF_ATHLETES_2013,APR_RATE_2012_1000,ELIG_RATE_2012,RET_RATE_2012,NUM_OF_ATHLETES_2012,APR_RATE_2011_1000,ELIG_RATE_2011,RET_RATE_2011,NUM_OF_ATHLETES_2011,APR_RATE_2010_1000,ELIG_RATE_2010,RET_RATE_2010,NUM_OF_ATHLETES_2010,APR_RATE_2009_1000,ELIG_RATE_2009,RET_RATE_2009,NUM_OF_ATHLETES_2009,APR_RATE_2008_1000,ELIG_RATE_2008,RET_RATE_2008,NUM_OF_ATHLETES_2008,APR_RATE_2007_1000,ELIG_RATE_2007,RET_RATE_2007,NUM_OF_ATHLETES_2007,APR_RATE_2006_1000,ELIG_RATE_2006,RET_RATE_2006,NUM_OF_ATHLETES_2006,APR_RATE_2005_1000,ELIG_RATE_2005,RET_RATE_2005,NUM_OF_ATHLETES_2005,APR_RATE_2004_1000,ELIG_RATE_2004,RET_RATE_2004,NUM_OF_ATHLETES_2004,PUB_AWARD_20,PUB_AWARD_19,PUB_AWARD_18,PUB_AWARD_17,PUB_AWARD_16,PUB_AWARD_15,PUB_AWARD_14,PUB_AWARD_13,PUB_AWARD_12,PUB_AWARD_11,PUB_AWARD_10,PUB_AWARD_09,PUB_AWARD_08,PUB_AWARD_07,PUB_AWARD_06
0,100654,Alabama A&M University,1,Baseball,2019,1,2,Southwestern Athletic Conf.,Y,N,923.0,948.0,923.0,raw,81.0,0.900621,0.914474,947.0,0.947368,0.945946,19.0,907.0,0.909091,0.904762,22.0,920.0,0.897436,0.916667,20.0,922.0,0.850000,0.891892,20.0,798.0,0.714286,0.857143,21.0,845.0,0.714286,0.976190,21.0,838.0,0.729730,0.945946,21.0,905.0,0.875000,0.914894,25.0,957.0,0.916667,1.000000,12.0,906.0,0.875000,0.937500,16.0,969.0,0.843750,1.000000,16.0,938.0,0.937500,0.937500,16.0,953.0,0.906250,1.000000,16.0,923.0,0.897436,0.948718,21.0,917.0,0.880952,0.952381,21.0,975.0,0.97500,0.975000,20.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100654,Alabama A&M University,2,Men's Basketball,2019,1,2,Southwestern Athletic Conf.,Y,N,908.0,941.0,908.0,raw,49.0,0.870968,0.923077,867.0,0.875000,0.857143,13.0,976.0,0.900000,1.000000,11.0,870.0,0.826087,0.913043,12.0,923.0,0.884615,0.923077,13.0,886.0,0.722222,1.000000,9.0,854.0,0.708333,1.000000,12.0,850.0,0.650000,1.000000,10.0,1000.0,1.000000,1.000000,8.0,940.0,0.880000,0.960000,13.0,889.0,0.956522,0.818182,12.0,844.0,0.875000,0.812500,8.0,915.0,0.916667,0.913043,12.0,964.0,0.964286,0.964286,14.0,923.0,0.961538,0.846154,13.0,909.0,0.909091,0.909091,11.0,950.0,0.95000,0.950000,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100654,Alabama A&M University,4,Football,2019,1,2,Southwestern Athletic Conf.,Y,N,924.0,938.0,924.0,raw,316.0,0.900515,0.932759,900.0,0.889655,0.909722,78.0,944.0,0.923077,0.936620,76.0,942.0,0.912162,0.952381,82.0,912.0,0.877551,0.931973,80.0,881.0,0.793548,0.948387,82.0,833.0,0.666667,0.979592,78.0,857.0,0.754839,0.954248,86.0,946.0,0.906667,0.959732,78.0,951.0,0.936620,0.950704,79.0,931.0,0.916031,0.937500,69.0,935.0,0.909091,0.946565,74.0,930.0,0.901639,0.950000,64.0,919.0,0.859060,0.979866,75.0,941.0,0.911111,0.962963,70.0,921.0,0.875862,0.965517,74.0,871.0,0.86014,0.881119,74.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100654,Alabama A&M University,6,Men's Golf,2019,1,2,Southwestern Athletic Conf.,Y,N,896.0,946.0,946.0,ci,21.0,0.871795,0.921053,962.0,1.000000,0.923077,7.0,1000.0,1.000000,1.000000,4.0,NaN,NaN,NaN,NaN,739.0,0.666667,0.818182,7.0,778.0,0.777778,0.777778,5.0,1000.0,0.875000,1.000000,4.0,833.0,0.777778,0.888889,5.0,750.0,0.700000,0.800000,6.0,750.0,0.750000,0.750000,7.0,NaN,NaN,NaN,NaN,857.0,0.857143,0.857143,4.0,1000.0,0.900000,1.000000,5.0,1000.0,1.000000,1.000000,4.0,889.0,

The columns remaining are SCL_UNITID (int) which is the id for each column, SCL_NAME (character), the name for the university, Sport_Code (Cateogry) and Sport_Name (chategory) which represent the id and name of the respective sport, Academic_Year(category) the year for that row, SCL_DIV_19 (Category), and SCL_SUB_19 (Category) which represent the NCAA division and subdivision they play in. CONFNAME (character) is the schools conference they compete in. SL_HBCU (category) and SL_Private (category) which is a binary data that shows if a school is HBCU and if a school is private. Then there are MULTIYR_APR_RATE_1000_RAW, MULTIYR_APR_RATE_1000_CI, MULTIYR_APR_RATE_1000_OFFICIAL, MULTIYR_SQUAD_SIZE, MULTIYR_ELIG_RATE,MULTIYR_RET_RATE which are all numeric and represent multiyear academic stats. Then from each year from 2006-2019 there are APR_RATE_XXXX_1000, ELIG_RATE_XXXX, RET_RATE_XXX, NUM_OF_ATHLETES_XXXX which are all numeric and represent single year academic stats. There are a lot of each column these are which each category of column means
APR_Rate: Academic Progress Rate as calculated by the NCAA.
ELIG_RATE: Percentage of athletes on a team which are academically elligible to compete.
RET_RATE: Percenentage of athletes who remain on the team after a year (excluding gratuating seniors and transfers)
NUM_OF_ATHLETES/MULTIYR_SQUAD_SIZE: The number of athletes on a sports teams roster.

Finally from 2006-2020 there are PUB_AWARD_XX which are categorical values that display if a school got an academic award from the NCAA.